In [456]:
import pandas as pd
import numpy as np
import statsmodels as sm
import sympy as sp
import matplotlib.pyplot as plt
import cvxpy as cvx
from cvxpy import *

In [1006]:
df_dict = {}
for i in range(1,11):
    home = 'Home'+str(i)
    file_name = './Data/'+home+'_yr1.csv'
    df_dict[home] = pd.read_csv(file_name, names = ['provision'])
    df_dict[home]['timestamp'] = pd.date_range(start = '12/1/2014', end = '12/1/2015', freq = '15T')[:-1]
    df_dict[home] = df_dict[home][['timestamp', 'provision']]
    df_dict[home].index = df_dict[home].timestamp

## Prediction models
Using two prediction models:
1. Moving Average Forecast (maf)
2. Simple Exponential Smoothing (ses)

In [1008]:
# #1. Moving Average Forecast
# for i in range(1,11):
#     home = 'Home'+str(i)
#     df_dict[home]['maf'] = df_dict[home]['provision'].rolling(96).mean()

In [1009]:
# #2. Simple Exponential Smoothing
# from statsmodels.tsa.holtwinters import ExponentialSmoothing
# for i in range(1,11):
#     home = 'Home'+str(i)
#     test = 0.2 #Use 20% data for testing prediction
#     split = int((1-test_split) * len(df_dict[home]))
#     train = df_dict[home][:split]
#     test = df_dict[home][split:]
#     model = ExponentialSmoothing(train['provision'], seasonal='mul', seasonal_periods=96).fit()
#     df_dict[home]['ses'] = model.predict(start=test.index[0], end=test.index[-1])

In [1034]:
udf = {}
pre = pred = {}
for i in range(1, 11):
    home = 'Home'+str(i)
    start_date = '11-01-2015'
    end_date = '11-08-2015'
    mask = (df_dict[home]['timestamp'] >= start_date) & (df_dict[home]['timestamp'] < end_date)
    udf[home] = df_dict[home].loc[mask]
    file_name = './Data/'+home+'.pkl'
    pre[home] = pd.read_pickle(file_name)
    start_date = '2015-11-01 00:00:00'
    end_date = '2015-11-08 02:30:00'
    mask = (pred[home]['timestamp'] >= start_date) & (pred[home]['timestamp'] < end_date)
    pred[home] = pre[home].loc[mask]

### Objective function

In [1063]:
def costObjFn(x,y,p,a,b,n=None):
    res = 0
    if not n:
        for t in range(len(y)):
            res += (p*x[t] + a*np.maximum(0,y[t]-x[t]) + b*np.absolute(x[t]-x[t-1]))
    else:
        for t in range(n):
            res += (p*x[t] + a*np.maximum(0,y[t]-x[t]) + b*np.absolute(x[t]-x[t-1]))
    return res

In [1064]:
def cvxDynamicObjFn(x,y,p,a,b):
    F = 0
    for t in range(len(y)):
        F += (multiply(p,x[t]) + multiply(a,maximum(0,y[t]-x[t])) + multiply(b,abs(x[t]-x[t-1])))
    return F

## 1. Offline Optimization Algorithms

### Offline Static

In [1082]:
def offlineStatic(y,p,a):
    def cvxStaticObjFn(x,y,p,a):
        F = 0
        for t in range(len(y)):
            F += (multiply(p,x) + multiply(a,maximum(0,y[t]-x)))
        return F

    x = cvx.Variable(1)
    objective = cvx.Minimize(cvxStaticObjFn(x,y,p,a))
    constraints = [x >= 0]
    prob = cvx.Problem(objective, constraints)
    res = prob.solve(solver=cvx.SCS)
    return res

### Offline Dynamic

In [1066]:
def offlineDynamic(y,p,a,b):
    x = cvx.Variable(len(y))
    objective = cvx.Minimize(cvxDynamicObjFn(x,y,p,a,b))
    constraints = [x >= 0]
    prob = cvx.Problem(objective, constraints)
    prob.solve(solver=cvx.SCS)
    return x.value

## 2. Online Optimization Algorithms

### Online Gradient Descent

In [1067]:
def ogd(y,s,p,a,b):
    x = [0]*len(y)
    def gradient(t):
        res = p
        if y[t] > x[t]:
            if x[t] < x[t-1]:
                res-=(a+b)
        elif y[t] <= x[t]:
            if x[t] < x[t-1]:
                res-=b
            elif x[t] > x[t-1]:
                res+=b
        return res

    for t in range(len(y)-1):
        ss = s
        if t>0:
            ss/=np.sqrt(t)
        x[t+1] = x[t] - ss*(gradient(t))
    return x

### Receding Horizon Control

In [1068]:
def rhc(y,n,w,p,a,b):
    x = [0]*n
    for t in range(n):
        x_rhc = cvx.Variable(w)
        y_hat = y[t:t+w]
        objective = cvx.Minimize(cvxDynamicObjFn(x_rhc,y_hat,p,a,b))
        constraints = [x_rhc >= 0]
        prob = cvx.Problem(objective, constraints)
        prob.solve(solver=cvx.SCS)
        x[t] = x_rhc.value[0]
    return x

### Commitment Horizon Control

In [1076]:
def chc(y,n,w,v,p,a,b):
    x = [0]*n
    for t in range(n):
        x_rhc = cvx.Variable(w)
        y_hat = y[t:t+w]
        objective = cvx.Minimize(cvxDynamicObjFn(x_rhc,y_hat,p,a,b))
        constraints = [x_rhc >= 0]
        prob = cvx.Problem(objective, constraints)
        prob.solve(solver=cvx.SCS)
        for i in range(v):
            if t+i<n:
                val = x_rhc.value[i]
                x[t+i] += val
    for i in range(v-1,n):
        x[i]/=v
    for i in range(1,v-1):
        x[i]/=(i+1)
    return x

# Main iteration

In [1131]:
step_sizes = [0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 1, 2, 5, 10]
window_sizes = [3, 5, 7, 10, 12, 15]
w_chc = [5,10,15]
commitment_levels = [[2, 4],[3,6,9],[4,8,12]]

off_cols = ['Offline Static', 'Offline Dynamic']
ogd_cols = ['OGD ' + str(step_size) for step_size in step_sizes]
rhc_cols_maf = ['MAF_RHC ' + str(window_size) for window_size in window_sizes]
rhc_cols_ses = ['SES_RHC ' + str(window_size) for window_size in window_sizes]
chc_cols_maf = []
chc_cols_ses = []
for l in range(len(w_chc)):
    for commitment_level in commitment_levels[l]:
        chc_cols_maf += ['MAF_CHC_W_' + str(w_chc[l]) + '_V_' + str(commitment_level)]
        chc_cols_ses += ['SES_CHC_W_' + str(w_chc[l]) + '_V_' +  str(commitment_level)]

cols = off_cols + ogd_cols + rhc_cols_maf + rhc_cols_ses + chc_cols_maf + chc_cols_ses
main_df = pd.DataFrame(columns=cols)
# main_df = []

a = 1   # penalty
b = 1   # switching cost
p = 0.1 # p is the electricity price
n = 672
for i in range(1,11):
    home = 'Home'+str(i)

    y = np.asarray(udf[home]['provision'])
#     df = pd.DataFrame(columns=cols)
    df = {}
    #'OGD10','RHC5', 'CHC3'
    print(home)
    # ------------Offine static-----------------

    cost = offlineStatic(y,p,a)
    # Assign cost
    df['Offline Static'] = cost
    print("Offline Static: ",cost)

    # -----------Offline dynamic----------------

    # Do computation here...
    x = offlineDynamic(y,p,a,b)
    udf[home]['Offline Dynamic'] =  x
    cost = costObjFn(x,y,p,a,b)
    # Assign cost
    df['Offline Dynamic'] = cost
    print("Offline Dynamic: ",cost)

    # ----------------OGD------------------------
    for step_size in step_sizes:
        col_name = 'OGD ' + str(step_size)
        x = ogd(y,step_size,p,a,b)
        udf[home][col_name] =  x
        cost = costObjFn(x,y,p,a,b)
        # Assign data to column
        df[col_name] = cost
        print(col_name,": ",cost)

    # ---------------RHC------------------------
#     y1 = np.asarray(pred[home]['moving_avg_forecast'])
#     for window_size in window_sizes:
#         col_name = 'MAF_RHC ' + str(window_size)
#         x = rhc(y1,n,window_size,p,a,b)
#         udf[home][col_name] =  x
#         cost = costObjFn(x,y,p,a,b,n)
#         # Assign data to column
#         df[col_name] = cost
#         print(col_name,": ",cost)
    
#     y2 = np.asarray(pred[home]['SES'])
#     for window_size in window_sizes:
#         col_name = 'SES_RHC ' + str(window_size)
#         x = rhc(y2,n,window_size,p,a,b)
#         udf[home][col_name] =  x
#         cost = costObjFn(x,y,p,a,b,n)
#         # Assign data to column
#         df[col_name] = cost
#         print(col_name,": ",cost)

#     # ---------------CHC-------------------------
#     for l in range(len(w_chc)):
#         for commitment_level in commitment_levels[l]:
#             w = w_chc[l]
#             col_name = 'MAF_CHC_W_' + str(w) + '_V_' + str(commitment_level)
#             x = chc(y1,n,w,commitment_level,p,a,b)
#             udf[home][col_name] =  x
#             cost = costObjFn(x,y,p,a,b,n)
#             # Assign data to column
#             df[col_name] = cost
#             print(col_name,": ",cost)

#     for l in range(len(w_chc)):
#         for commitment_level in commitment_levels[l]:
#             w = w_chc[l]
#             col_name = 'SES_CHC_W_' + str(w) + '_V_' +  str(commitment_level)
#             x = chc(y2,n,w,commitment_level,p,a,b)
#             udf[home][col_name] =  x
#             cost = costObjFn(x,y,p,a,b,n)
#             # Assign data to column
#             df[col_name] = cost
#             print(col_name,": ",cost)
        
    main_df = main_df.append(df, ignore_index = True)

Home1
Offline Static:  228.42657966739472


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Offline Dynamic:  197.3608134277959
OGD 0.01 :  542.0219843335702
OGD 0.05 :  357.4122291437306
OGD 0.1 :  299.06099909729727
OGD 0.2 :  261.6226511648967
OGD 0.3 :  259.00964780808704
OGD 0.5 :  263.88341607614313
OGD 1 :  285.92220976600817
OGD 2 :  337.57822525215096
OGD 5 :  504.47856916023295
OGD 10 :  783.0277519966921
Home2
Offline Static:  266.4337483478622
Offline Dynamic:  225.7705622902516
OGD 0.01 :  590.1765051416265
OGD 0.05 :  403.1695126220793
OGD 0.1 :  349.5020600633328
OGD 0.2 :  320.27987083765106
OGD 0.3 :  311.28124408691383
OGD 0.5 :  305.96853578562417
OGD 1 :  324.60264660313516
OGD 2 :  380.1426522987704
OGD 5 :  537.6794679228584
OGD 10 :  820.0910012138511
Home3
Offline Static:  231.97188037413378
Offline Dynamic:  197.80384539231753
OGD 0.01 :  555.1117372850454
OGD 0.05 :  376.49319286210516
OGD 0.1 :  311.59679248501374
OGD 0.2 :  269.36177121957087
OGD 0.3 :  263.343442679173
OGD 0.5 :  267.81365367991486
OGD 1 :  290.36025768953084
OGD 2 :  337.52139825

In [ ]:
udf

In [1140]:
cost_df = pd.read_pickle('./Data/cost_df.pkl')

In [1175]:
print(cost_df['Offline Dynamic'])

0    197.359744
1    225.771146
2    197.803843
3    165.432662
4    208.300169
5    244.091041
6    245.476443
7    249.096099
8    183.014200
9    169.430071
Name: Offline Dynamic, dtype: float64


In [1124]:
urdf = pd.read_pickle('./Data/udf.pkl')

In [1139]:
step_sizes = [0.5]
window_sizes = [15]
w_chc = [15]
commitment_levels = [[4,8,12]]

off_cols = ['Offline Static', 'Offline Dynamic']
ogd_cols = ['OGD ' + str(step_size) for step_size in step_sizes]
rhc_cols_maf = ['MAF_RHC ' + str(window_size) for window_size in window_sizes]
rhc_cols_ses = ['SES_RHC ' + str(window_size) for window_size in window_sizes]
chc_cols_maf = []
chc_cols_ses = []
for l in range(len(w_chc)):
    for commitment_level in commitment_levels[l]:
        chc_cols_maf += ['MAF_CHC_W_' + str(w_chc[l]) + '_V_' + str(commitment_level)]
        chc_cols_ses += ['SES_CHC_W_' + str(w_chc[l]) + '_V_' +  str(commitment_level)]

cols = off_cols + ogd_cols + rhc_cols_maf + rhc_cols_ses + chc_cols_maf + chc_cols_ses
main_df = pd.DataFrame(columns=cols)

avals = [0.1, 0.5, 2, 10]
b = 1   # switching cost
p = 0.1 # p is the electricity price
n = 672
home = 'Home6'

for a in avals:
    abdf = {}
    y = np.asarray(udf['Home6']['provision'])
    #     df = pd.DataFrame(columns=cols)
    #'OGD10','RHC5', 'CHC3'
    print(home)
    # ------------Offine static-----------------

    cost = offlineStatic(y,p,a)
    # Assign cost
    abdf['Offline Static'] = cost
    print("Offline Static: ",cost)

    # -----------Offline dynamic----------------

    # Do computation here...
    x = offlineDynamic(y,p,a,b)
    cost = costObjFn(x,y,p,a,b)
    # Assign cost
    abdf['Offline Dynamic'] = cost
    print("Offline Dynamic: ",cost)

    # ----------------OGD------------------------
    for step_size in step_sizes:
        col_name = 'OGD ' + str(step_size)
        x = ogd(y,step_size,p,a,b)
        cost = costObjFn(x,y,p,a,b)
        # Assign data to column
        abdf[col_name] = cost
        print(col_name,": ",cost)

    # ---------------RHC------------------------
    y1 = np.asarray(pred['Home6']['moving_avg_forecast'])
    for window_size in window_sizes:
        col_name = 'MAF_RHC ' + str(window_size)
        x = rhc(y1,n,window_size,p,a,b)
        cost = costObjFn(x,y,p,a,b,n)
        # Assign data to column
        abdf[col_name] = cost
        print(col_name,": ",cost)

    y2 = np.asarray(pred['Home6']['SES'])
    for window_size in window_sizes:
        col_name = 'SES_RHC ' + str(window_size)
        x = rhc(y2,n,window_size,p,a,b)
        cost = costObjFn(x,y,p,a,b,n)
        # Assign data to column
        abdf[col_name] = cost
        print(col_name,": ",cost)

    # ---------------CHC-------------------------
    for l in range(len(w_chc)):
        for commitment_level in commitment_levels[l]:
            w = w_chc[l]
            col_name = 'MAF_CHC_W_' + str(w) + '_V_' + str(commitment_level)
            x = chc(y1,n,w,commitment_level,p,a,b)
            cost = costObjFn(x,y,p,a,b,n)
            # Assign data to column
            abdf[col_name] = cost
            print(col_name,": ",cost)

    for l in range(len(w_chc)):
        for commitment_level in commitment_levels[l]:
            w = w_chc[l]
            col_name = 'SES_CHC_W_' + str(w) + '_V_' +  str(commitment_level)
            x = chc(y2,n,w,commitment_level,p,a,b)
            cost = costObjFn(x,y,p,a,b,n)
            # Assign data to column
            abdf[col_name] = cost
            print(col_name,": ",cost)
    main_df = main_df.append(abdf, ignore_index = True)

Home6
Offline Static:  65.65811049023931
Offline Dynamic:  65.99841387042439
OGD 0.5 :  132.1164866259313
MAF_RHC 15 :  85.78211422827756
SES_RHC 15 :  74.5449858165265
MAF_CHC_W_15_V_4 :  77.41978272217943
MAF_CHC_W_15_V_8 :  76.01125938520369
MAF_CHC_W_15_V_12 :  75.63989944919372
SES_CHC_W_15_V_4 :  72.45721912037183
SES_CHC_W_15_V_8 :  71.97402009968874
SES_CHC_W_15_V_12 :  71.65188989735017
Home6
Offline Static:  224.07655573288392
Offline Dynamic:  202.56030239028624
OGD 0.5 :  252.19765940411722
MAF_RHC 15 :  227.1848939935785
SES_RHC 15 :  240.48476337664943
MAF_CHC_W_15_V_4 :  226.76723811858815
MAF_CHC_W_15_V_8 :  226.60434255224288
MAF_CHC_W_15_V_12 :  226.57161729811094
SES_CHC_W_15_V_4 :  238.81097363162465
SES_CHC_W_15_V_8 :  237.67952162330602
SES_CHC_W_15_V_12 :  236.605028878954
Home6
Offline Static:  350.77735933704497
Offline Dynamic:  263.4819812050537
OGD 0.5 :  440.1119700664594
MAF_RHC 15 :  690.2912030171038
SES_RHC 15 :  778.6636990101397
MAF_CHC_W_15_V_4 :  68

In [1144]:
bvals = [0.1, 0.5, 2, 10]
a = 1  
p = 0.1 # p is the electricity price
n = 672
home = 'Home6'

for b in bvals:
    abdf = {}
    y = np.asarray(udf['Home6']['provision'])
    #     df = pd.DataFrame(columns=cols)
    #'OGD10','RHC5', 'CHC3'
    print(home)
    # ------------Offine static-----------------

    cost = offlineStatic(y,p,a)
    # Assign cost
    abdf['Offline Static'] = cost
    print("Offline Static: ",cost)

    # -----------Offline dynamic----------------

    # Do computation here...
    x = offlineDynamic(y,p,a,b)
    cost = costObjFn(x,y,p,a,b)
    # Assign cost
    abdf['Offline Dynamic'] = cost
    print("Offline Dynamic: ",cost)

    # ----------------OGD------------------------
    for step_size in step_sizes:
        col_name = 'OGD ' + str(step_size)
        x = ogd(y,step_size,p,a,b)
        cost = costObjFn(x,y,p,a,b)
        # Assign data to column
        abdf[col_name] = cost
        print(col_name,": ",cost)

    # ---------------RHC------------------------
    y1 = np.asarray(pred['Home6']['moving_avg_forecast'])
    for window_size in window_sizes:
        col_name = 'MAF_RHC ' + str(window_size)
        x = rhc(y1,n,window_size,p,a,b)
        cost = costObjFn(x,y,p,a,b,n)
        # Assign data to column
        abdf[col_name] = cost
        print(col_name,": ",cost)

    y2 = np.asarray(pred['Home6']['SES'])
    for window_size in window_sizes:
        col_name = 'SES_RHC ' + str(window_size)
        x = rhc(y2,n,window_size,p,a,b)
        cost = costObjFn(x,y,p,a,b,n)
        # Assign data to column
        abdf[col_name] = cost
        print(col_name,": ",cost)

    # ---------------CHC-------------------------
    for l in range(len(w_chc)):
        for commitment_level in commitment_levels[l]:
            w = w_chc[l]
            col_name = 'MAF_CHC_W_' + str(w) + '_V_' + str(commitment_level)
            x = chc(y1,n,w,commitment_level,p,a,b)
            cost = costObjFn(x,y,p,a,b,n)
            # Assign data to column
            abdf[col_name] = cost
            print(col_name,": ",cost)

    for l in range(len(w_chc)):
        for commitment_level in commitment_levels[l]:
            w = w_chc[l]
            col_name = 'SES_CHC_W_' + str(w) + '_V_' +  str(commitment_level)
            x = chc(y2,n,w,commitment_level,p,a,b)
            cost = costObjFn(x,y,p,a,b,n)
            # Assign data to column
            abdf[col_name] = cost
            print(col_name,": ",cost)
    main_df = main_df.append(abdf, ignore_index = True)

Home6
Offline Static:  302.09709978753665
Offline Dynamic:  108.42564076744947
OGD 0.5 :  346.26299085394123
MAF_RHC 15 :  382.3732983406451
SES_RHC 15 :  423.6315657444992
MAF_CHC_W_15_V_4 :  383.13888254619684
MAF_CHC_W_15_V_8 :  383.3487924100848
MAF_CHC_W_15_V_12 :  383.4195881080155
SES_CHC_W_15_V_4 :  423.0931681328826
SES_CHC_W_15_V_8 :  423.0038718026442
SES_CHC_W_15_V_12 :  423.00417364698245
Home6
Offline Static:  302.09709985992373
Offline Dynamic:  201.94061909927922
OGD 0.5 :  351.18819790743294
MAF_RHC 15 :  382.09481204800323
SES_RHC 15 :  423.1131121078839
MAF_CHC_W_15_V_4 :  381.9073637260617
MAF_CHC_W_15_V_8 :  381.99693937331216
MAF_CHC_W_15_V_12 :  382.33077673682476
SES_CHC_W_15_V_4 :  421.54326441690074
SES_CHC_W_15_V_8 :  420.2055590161582
SES_CHC_W_15_V_12 :  418.99251703644245
Home6
Offline Static:  302.0970997403422
Offline Dynamic:  290.5397708268659
OGD 0.5 :  410.77989089271983
MAF_RHC 15 :  385.424427970278
SES_RHC 15 :  431.2676215182652
MAF_CHC_W_15_V_4 

In [1173]:
# main_df.to_pickle('./ab.pkl')
main_df

,Offline Static,Offline Dynamic,OGD 0.5,MAF_RHC 15,SES_RHC 15,MAF_CHC_W_15_V_4,MAF_CHC_W_15_V_8,MAF_CHC_W_15_V_12,SES_CHC_W_15_V_4,SES_CHC_W_15_V_8,SES_CHC_W_15_V_12
0,65.658110,65.998414,132.116487,85.782114,74.544986,77.419783,76.011259,75.639899,72.457219,71.974020,71.651890
1,224.076556,202.560302,252.197659,227.184894,240.484763,226.767238,226.604343,226.571617,238.810974,237.679522,236.605029
2,350.777359,263.481981,440.111970,690.291203,778.663699,689.814144,688.931921,688.577127,775.451728,772.494076,770.088384
3,404.297024,264.810181,453.975592,3161.585255,3655.098399,3159.635535,3155.695109,3154.331831,3640.855630,3627.762735,3617.512459
4,302.097100,108.425641,346.262991,382.373298,423.631566,383.138883,383.348792,383.419588,423.093168,423.003872,423.004174
5,302.097100,201.940619,351.188198,382.094812,423.113112,381.907364,381.996939,382.330777,421.543264,420.205559,418.992517
6,302.097100,290.539771,410.779891,385.424428,431.267622,385.191562,384.681602,384.400617,430.077917,427.606851,425.436936
7,302.097100,308.003332,2885.706551,412.747759,487.773775,412.308742,410.730530,409.273788,486.291595,478.946367,472.619151


In [1155]:
abdf

{'Offline Static': 302.09709986906455,
 'Offline Dynamic': 308.00333232846543,
 'OGD 0.5': 2885.7065512571485,
 'MAF_RHC 15': 412.7477588147792,
 'SES_RHC 15': 487.7737751961456,
 'MAF_CHC_W_15_V_4': 412.3087415172171,
 'MAF_CHC_W_15_V_8': 410.730530496107,
 'MAF_CHC_W_15_V_12': 409.27378764033364,
 'SES_CHC_W_15_V_4': 486.29159488093745,
 'SES_CHC_W_15_V_8': 478.9463667204604,
 'SES_CHC_W_15_V_12': 472.61915100733324}

### Algorithm Selection

### Deterministic

In [1170]:
interval = 12
p = 0.1
a = b = 1
n = 672
i=0
cost = 0
ss = 1
window = 5
commitment = 3
y = np.asarray(udf['Home6']['provision'])
yp1 = np.asarray(pred['Home6']['moving_avg_forecast'])
count = 1
decisions = {}
decision = 1
algos = {1:'Online Gradient Descent', 
         2:'Receding Horizon Control', 
         3:'Commitment Horizon Control'}

while (i<n):
    # Offline Dynamic
    print(i)
        
    costv = []
    yv = y[i:i+interval]
    ypv = yp1[i:i+interval+window]
    
    x2 = ogd(yv,ss,p,a,b)
    costv.append(costObjFn(x2,yv,p,a,b))
    
    x3 = rhc(ypv,interval,window,p,a,b)
    costv.append(costObjFn(x3,yv,p,a,b,interval))
    
    x4 = chc(ypv,n,window,commitment,p,a,b)
    costv.append(costObjFn(x4,yv,p,a,b,interval))
    
    mincost = 99999999
    minelem = 0
    for k in range(len(costv)):
        if mincost > costv[k]:
            mincost = costv[k]
            minelem = k
    
    cost+=mincost
    decisions[decision] = algos[minelem+1]
    print(cost, decisions[decision])
    i+=interval
    decision+=1

print(cost, decisions)

0
7.721302073311415 Receding Horizon Control
12
10.155855457459003 Receding Horizon Control
24
17.245875587838626 Receding Horizon Control
36
25.890412360217375 Receding Horizon Control
48
37.93673689355184 Receding Horizon Control
60
44.53816649125952 Receding Horizon Control
72
54.37380948387391 Receding Horizon Control
84
61.92591509465912 Receding Horizon Control
96
63.55626610321046 Receding Horizon Control
108
65.02194379599204 Receding Horizon Control
120
74.207098018287 Receding Horizon Control
132
81.78345238109294 Receding Horizon Control
144
90.36769678234171 Receding Horizon Control
156
98.32184688746534 Receding Horizon Control
168
100.26825597217258 Receding Horizon Control
180
106.46579581470259 Receding Horizon Control
192
107.61066421155593 Receding Horizon Control
204
108.81265074502429 Receding Horizon Control
216
117.87630531137837 Receding Horizon Control
228
121.50013389195266 Receding Horizon Control
240
131.80564312404636 Receding Horizon Control
252
134.3388270

### Randomized

In [ ]:
intervals = [4,8,12,16,21,24,32,48]
from random import randint
l = randint(1,len(intervals))
interval = intervals[l-1]
b = randint(1,3)